#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [2]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [3]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [5]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
12829,-10.0,77,5.5,56.0,overcast,995.79,0,Winter,4.0,mountain,Snowy
5587,13.0,78,8.0,59.0,overcast,1003.42,1,Autumn,1.0,inland,Rainy
11997,-8.0,82,9.5,73.0,overcast,991.80,0,Winter,1.5,inland,Snowy
695,4.0,85,13.0,95.0,overcast,988.30,0,Winter,1.5,inland,Snowy
4912,25.0,76,13.5,92.0,overcast,997.21,1,Winter,4.0,coastal,Rainy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [6]:
# START CODE HERE
object_columns = ['Cloud Cover', 'Season', 'Location', 'Weather Type']
non_object_columns = ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [7]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [8]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer = ColumnTransformer(
    transformers=[
        ('categorical', OrdinalEncoder(), object_columns),
        ('numerical', StandardScaler(), non_object_columns)
    ]
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [9]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [10]:
data_scaled_labeled.columns

Index(['Cloud Cover', 'Season', 'Location', 'Weather Type', 'Temperature',
       'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure',
       'UV Index', 'Visibility (km)'],
      dtype='object')

In [11]:
X = data_scaled_labeled[['Cloud Cover', 'Season', 'Location', 'Temperature',
       'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure',
       'UV Index', 'Visibility (km)']]
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [12]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [13]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,3.0,3.0,1.0,0.567848,-0.926578,-0.337587,-1.084490,0.189313,1.295035,0.307615
12278,0.0,3.0,2.0,0.567848,-0.678973,-0.409962,-1.115794,0.458143,0.776424,0.010999
2859,2.0,0.0,0.0,-0.294931,0.014320,0.675666,-0.771455,0.348998,-0.520104,1.049153
5495,2.0,2.0,0.0,-0.525006,0.113362,0.096665,-0.646241,0.326417,-0.779410,0.159307
8469,2.0,3.0,2.0,0.682886,-0.827536,0.386166,-0.990580,0.361633,-0.260799,0.752538


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [14]:
y.value_counts()

,count
Weather Type,
2.0,3300
1.0,3300
3.0,3300
0.0,3300


In [15]:
# START CODE HERE

from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1, random_state=42)
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 91.36363636363637


**TEST** (DON'T EDIT THIS CELL)

In [16]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [17]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,2.0,1.0,2.0,1.660703,0.261924,0.748042,0.574594,-2.157845,-0.001493,-0.878846
2331,2.0,0.0,2.0,-0.409968,0.063841,0.096665,0.605898,-0.322540,-0.520104,-1.323769
217,2.0,3.0,2.0,-1.157711,1.450427,0.603291,1.325878,-0.544056,-1.038715,-0.137308
1760,3.0,0.0,2.0,0.222737,-0.233285,-0.337587,-0.207993,0.257327,-0.001493,0.159307
3963,2.0,3.0,1.0,-1.100192,0.360966,-1.350840,0.543291,-0.259634,-1.038715,-1.175461


In [18]:
y = model.predict(input)
y

array([1., 1., 2., 0., 2.])

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
